# Intro
The purpose of this notebook is to prepare PACE (Propery Assessed Clean Energy loan data). The data contains reported loan data for addresses data in LA County, Califronia. People who agree to the program are often unaware that they are signing of for a loan which comes in the form of property taxes, and are approved by using the home value as collateral. I've been asked to visualize map poiunts based off geographic data and explore whether or not there is a concentration in Hispanic and Black populated zip codes. 

In [1]:
import fuzzywuzzy
import pandas as pd
import metaphone
import fuzzymatcher

# Working with Zipcodes

In [2]:
# Importing Data given to me
pace_data = pd.read_excel('2020.02.03_PACE_LA_County_Programs_Address_Match_Total.xlsx', sheet_name = 1)

In [3]:
# data set with zip_code
zip_codes = pd.read_csv('US_zip_codes.csv')

In [269]:
# use this to change row settings
# pd.set_option("display.max_rows", None)
pd.set_option('display.max_rows', 100)

In [5]:
pace_data.columns

Index(['Assessor's Parcel Number (APN)', 'Principal Amount',
       'Annual Assessment Amount', 'Maturity Date', 'Origination Date',
       'Program', 'Address', 'Zip', 'Count for Property', 'NOD', 'FC', 'Refi',
       'Unique Property', 'Address Missing'],
      dtype='object')

In [12]:
pace_data['Address'].value_counts(dropna = False)

NaN                                               2499
1120 STOVALL AVE  HACIENDA HEIGHTS CA  91745         7
18906 NEARBROOK ST  SANTA CLARITA CA  91351          7
10902 LOCH AVON DR  WHITTIER CA  90606               6
15385 RHODODENDRON DR  SANTA CLARITA CA  91387       6
                                                  ... 
632 S GRAND AVE  PASADENA CA  91105                  1
1073 E 8TH ST  POMONA CA  91766                      1
4601 LAKEWOOD BLVD  LAKEWOOD CA  90712               1
36855 FIDDLENECK CT  PALMDALE CA  93550              1
1877 W 38TH ST  LOS ANGELES CA  90062                1
Name: Address, Length: 26327, dtype: int64

In [11]:
pace_data['Zip'].value_counts(dropna = False)

NaN             2496
90650.000000     731
90047.000000     624
93535.000000     566
91331.000000     547
                ... 
0.923630           1
30175.000000       1
0.448625           1
90057.000000       1
90094.000000       1
Name: Zip, Length: 268, dtype: int64

In [9]:
# Only Zip code that doesn't start with 9
pace_data[pace_data['Zip'] == 30175]

,Assessor's Parcel Number (APN),Principal Amount,Annual Assessment Amount,Maturity Date,Origination Date,Program,Address,Zip,Count for Property,NOD,FC,Refi,Unique Property,Address Missing
32671,NaN,NaN,NaN,NaT,NaT,NaN,NaN,30175.0,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# filled null values merged zip_code data
pace_data['Address'] = pace_data['Address'].fillna('Unknown')

pace_data['Zip'] = pace_data['Zip'].fillna(0).astype(int)

pace_merged = pace_data.merge(zip_codes, how = 'left', left_on = 'Zip', right_on = 'ZIPCODE')

pace_merged = pace_merged[pace_merged['Address'] != 'Unknown']

In [45]:
pace_data.columns

Index(['Assessor's Parcel Number (APN)', 'Principal Amount',
       'Annual Assessment Amount', 'Maturity Date', 'Origination Date',
       'Program', 'Address', 'Zip', 'Count for Property', 'NOD', 'FC', 'Refi',
       'Unique Property', 'Address Missing'],
      dtype='object')

In [14]:
pace_merged = pace_merged.drop(['GEOID10', 'CLASSFP10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10',
       'AWATER10'], axis = 1)

NameError: name 'pace_merged' is not defined

In [13]:
pace_merged.head()

NameError: name 'pace_merged' is not defined

In [52]:
# personal data I don't want posted to tableau
pace_confidential = pace_merged.drop(["Assessor's Parcel Number (APN)", 'Address'],axis =1)

## Zipcode data export

In [53]:
# pace_confidential.to_csv('Pace_LA_County_Confidential')

# Working on Individual Level
### Found data on invidual level. Will merge data here to original dataset and then git rid of unwanted columns

In [4]:
all_parcel = pd.read_csv('Assessor_Parcels_Data_-_2006_thru_2021.csv')

In [5]:
pace_data.head()

,Assessor's Parcel Number (APN),Principal Amount,Annual Assessment Amount,Maturity Date,Origination Date,Program,Address,Zip,Count for Property,NOD,FC,Refi,Unique Property,Address Missing
0,2803-032-008,19421.99,2039.05,2036-09-02,2016-01-21,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,Y,NaN,NaN,1.0,0.0
1,2803-032-008,32185.60,4726.28,2026-09-02,2016-03-03,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,Y,NaN,NaN,0.0,0.0
2,2803-032-008,28074.74,2939.46,2036-09-02,2016-04-07,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,Y,NaN,NaN,0.0,0.0
3,2803-032-008,48148.36,4679.68,2041-09-02,2015-11-05,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,Y,NaN,NaN,0.0,0.0
4,2803-032-008,6827.82,815.20,2031-09-02,2015-12-28,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,Y,NaN,NaN,0.0,0.0


In [6]:
all_parcel.head()

,ZIPcode,TaxRateArea_CITY,AIN,RollYear,TaxRateArea,AssessorID,PropertyLocation,PropertyType,PropertyUseCode,GeneralUseType,...,HouseFraction,StreetDirection,StreetName,UnitNo,City,ZIPcode5,rowID,CENTER_LAT,CENTER_LON,Location 1
0,91604-3019,LOS ANGELES,2368011020,2020,13,2368-011-020,11571 DILLING ST LOS ANGELES CA 91604,VAC,010V,Residential,...,NaN,NaN,DILLING ST,NaN,LOS ANGELES CA,91604,20202368011020,34.143598,-118.384811,"(34.14359841, -118.38481081)"
1,90254-2453,HERMOSA BEACH,4181023010,2020,4340,4181-023-010,230 28TH ST HERMOSA BEACH CA 90254,SFR,0100,Residential,...,NaN,NaN,28TH ST,NaN,HERMOSA BEACH CA,90254,20204181023010,33.873747,-118.404461,"(33.87374743, -118.40446108)"
2,90039,LOS ANGELES,5431002033,2020,13,5431-002-033,1933 N GRIFFITH PARK BLVD LOS ANGELES CA 90039,SFR,01S0,Residential,...,NaN,N,GRIFFITH PARK BLVD,NaN,LOS ANGELES CA,90039,20205431002033,NaN,NaN,NaN
3,90703-1665,CERRITOS,7016028005,2020,11380,7016-028-005,11377 LAGOON CIR CERRITOS CA 90703,SFR,0100,Residential,...,NaN,NaN,LAGOON CIR,NaN,CERRITOS CA,90703,20207016028005,33.878671,-118.091474,"(33.87867104, -118.09147444)"
4,90814-2945,LONG BEACH,7250003003,2020,5500,7250-003-003,364 XIMENO AVE LONG BEACH CA 90814,SFR,0100,Residential,...,NaN,NaN,XIMENO AVE,NaN,LONG BEACH CA,90814,20207250003003,33.770341,-118.141240,"(33.77034108, -118.14123984)"


In [63]:
data = pace_data.merge(all_parcel, how = 'left', left_on = 'Address', right_on = 'PropertyLocation')

In [453]:
pace_data.describe()

,Principal Amount,Annual Assessment Amount,Zip,Unique Property,Address Missing
count,32670.000000,32670.000000,30178.000000,32671.000000,32671.000000
mean,27680.932502,3338.881885,91122.272794,1.748952,0.137370
std,18974.524646,2275.300049,1263.305522,158.058074,12.417051
min,1051.040000,466.800000,0.448625,0.000000,0.000000
25%,15479.895000,1908.057500,90302.000000,1.000000,0.000000
50%,22477.875000,2760.960000,91001.000000,1.000000,0.000000
75%,34139.180000,4124.912500,91607.000000,1.000000,0.000000
max,245183.290000,57439.960000,93591.000000,28570.000000,2244.000000


In [64]:
# List of Columns I don't need
drop_list = ['NOD', 'FC', 'Refi',
       'Unique Property', 'Address Missing', 'ZIPcode', 'TaxRateArea_CITY',
       'AIN', 'RollYear', 'TaxRateArea', 'AssessorID', 'PropertyLocation',
       'PropertyType', 'PropertyUseCode', 'GeneralUseType', 'SpecificUseType',
       'SpecificUseDetail1', 'SpecificUseDetail2', 'totBuildingDataLines',
       'YearBuilt', 'EffectiveYearBuilt', 'SQFTmain', 'Bedrooms', 'Bathrooms',
       'Units', 'RecordingDate', 'LandValue', 'LandBaseYear',
       'ImprovementValue', 'ImpBaseYear', 'TotalLandImpValue',
       'HomeownersExemption', 'RealEstateExemption', 'FixtureValue',
       'FixtureExemption', 'PersonalPropertyValue',
       'PersonalPropertyExemption', 'isTaxableParcel?', 'TotalValue',
       'TotalExemption', 'netTaxableValue', 'SpecialParcelClassification',
       'AdministrativeRegion', 'Cluster', 'ParcelBoundaryDescription',
       'HouseNo', 'HouseFraction', 'StreetDirection', 'StreetName', 'UnitNo',
       'City', 'ZIPcode5', 'rowID']

In [65]:
# Cleaning
data = data.drop(drop_list, axis = 1)

data = data.rename(columns = {'CENTER_LAT' : 'Latitude', 'CENTER_LON' : 'Longitude'})

In [66]:
data.head(10)

,Assessor's Parcel Number (APN),Principal Amount,Annual Assessment Amount,Maturity Date,Origination Date,Program,Address,Zip,Count for Property,Latitude,Longitude,Location 1,ID
0,2803-032-008,19421.99,2039.05,2036-09-02,2016-01-21,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,34.416684,-118.465632,"(34.41668389, -118.46563184)",0
1,2803-032-008,32185.60,4726.28,2026-09-02,2016-03-03,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,34.416684,-118.465632,"(34.41668389, -118.46563184)",1
2,2803-032-008,28074.74,2939.46,2036-09-02,2016-04-07,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,34.416684,-118.465632,"(34.41668389, -118.46563184)",2
3,2803-032-008,48148.36,4679.68,2041-09-02,2015-11-05,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,34.416684,-118.465632,"(34.41668389, -118.46563184)",3
4,2803-032-008,6827.82,815.20,2031-09-02,2015-12-28,CAFIRST LA,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,34.416684,-118.465632,"(34.41668389, -118.46563184)",4
5,2803-032-008,8656.69,904.78,2035-11-01,2015-09-16,LA HERO,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,34.416684,-118.465632,"(34.41668389, -118.46563184)",5
6,2803-032-008,44664.05,4668.25,2035-11-01,2015-10-28,LA HERO,18906 NEARBROOK ST SANTA CLARITA CA 91351,91351.0,7,34.416684,-118.465632,"(34.41668389, -118.46563184)",6
7,8218-019-028,10349.16,1081.69,2036-11-01,2016-09-28,LA HERO,1120 STOVALL AVE HACIENDA HEIGHTS CA 91745,91745.0,7,34.013691,-117.973091,"(34.01369074, -117.97309146)",7
8,8218-019-028,24271.56,2026.68,2036-11-01,2016-11-30,LA HERO,1120 STOVALL AVE HACIENDA HEIGHTS CA 91745,91745.0,7,34.013691,-117.973091,"(34.01369074, -117.97309146)",8
9,8218-019-028,10136.98,1072.85,2036-09-02,2016-07-14,CAFIRST LA,1120 STOVALL AVE HACIENDA HEIGHTS CA 91745,91745.0,7,34.013691,-117.973091,"(34.01369074, -117.97309146)",9


## Individual data export

In [72]:
# data.to_csv('Pace_Individual.csv')

In [73]:
data.isnull().sum()

Assessor's Parcel Number (APN)       4
Principal Amount                     4
Annual Assessment Amount             4
Maturity Date                        4
Origination Date                     4
Program                              4
Address                           2499
Zip                               2496
Count for Property                   2
Latitude                          2616
Longitude                         2616
Location 1                        2616
ID                                   0
dtype: int64

In [67]:
data ['ID'] = data['ID'].astype('object')

In [142]:
# Repeated data points need to be aggregated
summed = data.groupby("Assessor's Parcel Number (APN)").sum()

summed = summed.reset_index()

summed = summed.rename(columns = {'Principal Amount' : 'Total Principal', 'Annual Assessment Amount' : 'Annual Assessment Total'})

summed['index'] = summed.index.astype('object')

summed = summed[["Assessor's Parcel Number (APN)", 'Total Principal', 'Annual Assessment Total', 'index']]

In [143]:
summed

,Assessor's Parcel Number (APN),Total Principal,Annual Assessment Total,index
0,2004-005-007,26973.84,2819.29,0
1,2004-006-025,46338.93,4817.84,1
2,2004-007-013,18072.97,2130.84,2
3,2004-007-020,85973.15,9010.86,3
4,2004-009-014,18515.99,1509.06,4
...,...,...,...,...
28565,8765-010-033,20950.66,2189.74,28565
28566,8765-011-031,21310.33,2512.53,28566
28567,8765-012-004,31091.68,3249.68,28567
28568,8765-019-002,62447.82,7482.40,28568


In [70]:
summed.iloc[0,0]

'2004-005-007'

In [71]:
# checked if summation lined up 
summed[summed["Assessor's Parcel Number (APN)"] == '2803-032-008']

,Assessor's Parcel Number (APN),Principal Amount,Annual Assessment Amount,Zip,Latitude,Longitude
4897,2803-032-008,187979.25,20772.7,639457.0,240.916787,-829.259423


In [141]:
summed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28570 entries, 0 to 28569
Data columns (total 4 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Assessor's Parcel Number (APN)  28570 non-null  object 
 1   Total Principal                 28570 non-null  float64
 2   Annual Assessment Total         28570 non-null  float64
 3   index                           28570 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 892.9+ KB


In [ ]:
# adding aggregates to main dataset and dropping duplicates
data_grouped = data.merge(summed, how = 'left', left_on = "Assessor's Parcel Number (APN)", right_on = "Assessor's Parcel Number (APN)")

data_grouped = data_grouped.drop_duplicates(subset="Assessor's Parcel Number (APN)").sort_values('index').reindex()

In [155]:
len(data_grouped)

28571

In [162]:
# dropping non aggregate columns
non_group_columns = ['Principal Amount', 'Annual Assessment Amount', 'Origination Date', 'ID', 'Address']

data_grouped_confidential = data_grouped.drop(non_group_columns, axis = 1)

## Grouped Data Export

In [165]:
# data_grouped_confidential.to_csv('PACE_aggregate.csv')

# Population Data
### After talking to a census data specialist I acquired demographic data to detail my visualization. Here I import the data and prepare it for visualization.

In [20]:
import regex as re

In [21]:
black_pop = pd.read_csv('la_black_pop.csv')

In [22]:
black_pop.head()

,Label (Grouping),ZCTA5 90001!!Estimate,ZCTA5 90001!!Margin of Error,ZCTA5 90002!!Estimate,ZCTA5 90002!!Margin of Error,ZCTA5 90003!!Estimate,ZCTA5 90003!!Margin of Error,ZCTA5 90004!!Estimate,ZCTA5 90004!!Margin of Error,ZCTA5 90005!!Estimate,...,ZCTA5 93552!!Estimate,ZCTA5 93552!!Margin of Error,ZCTA5 93553!!Estimate,ZCTA5 93553!!Margin of Error,ZCTA5 93560!!Estimate,ZCTA5 93560!!Margin of Error,ZCTA5 93563!!Estimate,ZCTA5 93563!!Margin of Error,ZCTA5 93591!!Estimate,ZCTA5 93591!!Margin of Error
0,Total:,"5,338",±727,"10,081","±1,012","15,245","±1,187","2,188",±348,"2,290",...,"4,844",±922,18,±26,"1,828",±699,38,±28,544,±253
1,Male:,"2,636",±419,"4,080",±538,"6,749",±753,"1,272",±249,948,...,"2,415",±480,9,±13,761,±272,38,±28,265,±111
2,Under 5 years,112,±78,390,±154,720,±264,0,±29,86,...,280,±145,0,±12,89,±119,0,±12,0,±17
3,5 to 9 years,176,±92,192,±118,393,±148,0,±29,8,...,153,±82,0,±12,88,±101,0,±12,0,±17
4,10 to 14 years,171,±109,293,±126,556,±207,42,±53,0,...,61,±39,0,±12,57,±79,0,±12,0,±17


In [23]:
# Reorganizing the index and dropping extra columns/rows
black_totals = black_pop[black_pop['Label (Grouping)'] == 'Total:'].T.reset_index().iloc[1:,:]

black_totals.columns = ['subgroup_info', 'black_population']

# removing rows with margin of error calculations
black_totals = black_totals[black_totals['subgroup_info'].str.contains('Estimate')]

In [26]:
black_totals['subgroup_info']

1      ZCTA5 90001!!Estimate
3      ZCTA5 90002!!Estimate
5      ZCTA5 90003!!Estimate
7      ZCTA5 90004!!Estimate
9      ZCTA5 90005!!Estimate
               ...          
595    ZCTA5 93552!!Estimate
597    ZCTA5 93553!!Estimate
599    ZCTA5 93560!!Estimate
601    ZCTA5 93563!!Estimate
603    ZCTA5 93591!!Estimate
Name: subgroup_info, Length: 302, dtype: object

In [28]:
# Grabs the zipcodes from each string object in a list
# [(lambda a: re.findall(r'[0-9]{5}', a))(z)[0] for z in zip_codes[1:]]

In [29]:
# filtered out zipcodes from string
black_zct = [(lambda a: re.findall(r'[0-9]{5}', a))(z)[0] for z in black_totals['subgroup_info']]

In [30]:
# check length
print(len(black_totals['subgroup_info']),len(black_zct))

302 302


In [31]:
# replace string with only zipcode as numeric type
black_totals['zipcode'] = black_zct

black_totals['zipcode'] = black_totals['zipcode'].astype('float')

black_totals = black_totals.drop('subgroup_info', axis = 1)

In [34]:
black_totals

,black_population,zipcode
1,"5,338",90001.0
3,"10,081",90002.0
5,"15,245",90003.0
7,"2,188",90004.0
9,"2,290",90005.0
...,...,...
595,"4,844",93552.0
597,18,93553.0
599,"1,828",93560.0
601,38,93563.0


## Repeat process with hispanic data

In [35]:
hispanic_pop = pd.read_csv('la_hispanic_pop.csv')

In [36]:
hispanic_pop.head()

,Label (Grouping),ZCTA5 90001!!Estimate,ZCTA5 90001!!Margin of Error,ZCTA5 90002!!Estimate,ZCTA5 90002!!Margin of Error,ZCTA5 90003!!Estimate,ZCTA5 90003!!Margin of Error,ZCTA5 90004!!Estimate,ZCTA5 90004!!Margin of Error,ZCTA5 90005!!Estimate,...,ZCTA5 93552!!Estimate,ZCTA5 93552!!Margin of Error,ZCTA5 93553!!Estimate,ZCTA5 93553!!Margin of Error,ZCTA5 93560!!Estimate,ZCTA5 93560!!Margin of Error,ZCTA5 93563!!Estimate,ZCTA5 93563!!Margin of Error,ZCTA5 93591!!Estimate,ZCTA5 93591!!Margin of Error
0,Total:,"53,929","±1,790","42,048","±2,117","57,766","±2,208","30,446","±1,622","19,660",...,"28,313","±1,306",727,±307,"9,205","±1,032",59,±29,"4,227",±733
1,Male:,"28,044","±1,160","20,826","±1,178","29,099","±1,326","14,870","±1,031","10,427",...,"14,796",±938,307,±165,"4,889",±663,51,±27,"1,844",±367
2,Under 5 years,"2,378",±364,"2,006",±386,"2,703",±324,"1,066",±311,777,...,"1,169",±301,0,±12,171,±134,0,±12,127,±93
3,5 to 9 years,"2,921",±356,"2,133",±362,"2,829",±375,"1,022",±230,922,...,"1,499",±296,57,±34,612,±294,0,±12,186,±113
4,10 to 14 years,"2,679",±348,"2,021",±349,"2,946",±365,873,±222,728,...,"1,877",±330,94,±70,553,±225,0,±12,151,±89


In [37]:
hispanic_totals = hispanic_pop[hispanic_pop['Label (Grouping)'] == 'Total:'].T.reset_index().iloc[1:,:]

In [38]:
hispanic_totals.columns = ['subgroup_info', 'hispanic_population']

In [39]:
hispanic_totals = hispanic_totals[hispanic_totals['subgroup_info'].str.contains('Estimate')]

In [40]:
hispanic_totals['subgroup_info']

1      ZCTA5 90001!!Estimate
3      ZCTA5 90002!!Estimate
5      ZCTA5 90003!!Estimate
7      ZCTA5 90004!!Estimate
9      ZCTA5 90005!!Estimate
               ...          
595    ZCTA5 93552!!Estimate
597    ZCTA5 93553!!Estimate
599    ZCTA5 93560!!Estimate
601    ZCTA5 93563!!Estimate
603    ZCTA5 93591!!Estimate
Name: subgroup_info, Length: 302, dtype: object

In [41]:
# Grabs the zipcodes from each string object in a list
# [(lambda a: re.findall(r'[0-9]{5}', a))(z)[0] for z in zip_codes[1:]]

In [42]:
hispanic_zct = [(lambda a: re.findall(r'[0-9]{5}', a))(z)[0] for z in hispanic_totals['subgroup_info']]

In [43]:
# check length
print(len(hispanic_totals['subgroup_info']),len(hispanic_zct))

302 302


In [44]:
hispanic_totals['zipcode'] = hispanic_zct

In [45]:
hispanic_totals['zipcode'] = hispanic_totals['zipcode'].astype('float')

In [46]:
hispanic_totals = hispanic_totals.drop('subgroup_info', axis = 1)

In [47]:
hispanic_totals

,hispanic_population,zipcode
1,"53,929",90001.0
3,"42,048",90002.0
5,"57,766",90003.0
7,"30,446",90004.0
9,"19,660",90005.0
...,...,...
595,"28,313",93552.0
597,727,93553.0
599,"9,205",93560.0
601,59,93563.0


## Repeat process for total population

In [48]:
total_pop = pd.read_csv('la_total_pop.csv')

In [49]:
total_pop.head()

,Label (Grouping),ZCTA5 90001!!Estimate,ZCTA5 90001!!Margin of Error,ZCTA5 90001!!Percent,ZCTA5 90001!!Percent Margin of Error,ZCTA5 90002!!Estimate,ZCTA5 90002!!Margin of Error,ZCTA5 90002!!Percent,ZCTA5 90002!!Percent Margin of Error,ZCTA5 90003!!Estimate,...,ZCTA5 93560!!Percent,ZCTA5 93560!!Percent Margin of Error,ZCTA5 93563!!Estimate,ZCTA5 93563!!Margin of Error,ZCTA5 93563!!Percent,ZCTA5 93563!!Percent Margin of Error,ZCTA5 93591!!Estimate,ZCTA5 93591!!Margin of Error,ZCTA5 93591!!Percent,ZCTA5 93591!!Percent Margin of Error
0,SEX AND AGE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Total population,"59,832","±1,842","59,832",(X),"53,302","±2,254","53,302",(X),"73,730",...,"21,756",(X),357.0,±122,357,(X),"6,744",±779,"6,744",(X)
2,Male,"30,910","±1,173",51.7%,±1.1,"25,422","±1,302",47.7%,±1.2,"36,135",...,51.5%,±2.6,233.0,±78,65.3%,±7.9,"2,978",±391,44.2%,±3.4
3,Female,"28,922","±1,061",48.3%,±1.1,"27,880","±1,291",52.3%,±1.2,"37,595",...,48.5%,±2.6,124.0,±57,34.7%,±7.9,"3,766",±513,55.8%,±3.4
4,Sex ratio (males per 100 females),106.9,±4.5,(X),(X),91.2,±4.4,(X),(X),96.1,...,(X),(X),187.9,±68.0,(X),(X),79.1,±11.0,(X),(X)


In [50]:
total_pop.iloc[1,0] = 'Total population'

In [51]:
[total_pop['Label (Grouping)']]

[0                                   SEX AND AGE
 1                              Total population
 2                                          Male
 3                                        Female
 4             Sex ratio (males per 100 females)
                         ...                    
 89                          Total housing units
 90               CITIZEN, VOTING AGE POPULATION
 91              Citizen, 18 and over population
 92                                         Male
 93                                       Female
 Name: Label (Grouping), Length: 94, dtype: object]

In [52]:
total_pop[total_pop['Label (Grouping)'] == 'Total population']

,Label (Grouping),ZCTA5 90001!!Estimate,ZCTA5 90001!!Margin of Error,ZCTA5 90001!!Percent,ZCTA5 90001!!Percent Margin of Error,ZCTA5 90002!!Estimate,ZCTA5 90002!!Margin of Error,ZCTA5 90002!!Percent,ZCTA5 90002!!Percent Margin of Error,ZCTA5 90003!!Estimate,...,ZCTA5 93560!!Percent,ZCTA5 93560!!Percent Margin of Error,ZCTA5 93563!!Estimate,ZCTA5 93563!!Margin of Error,ZCTA5 93563!!Percent,ZCTA5 93563!!Percent Margin of Error,ZCTA5 93591!!Estimate,ZCTA5 93591!!Margin of Error,ZCTA5 93591!!Percent,ZCTA5 93591!!Percent Margin of Error
1,Total population,"59,832","±1,842","59,832",(X),"53,302","±2,254","53,302",(X),"73,730",...,"21,756",(X),357.0,±122,357,(X),"6,744",±779,"6,744",(X)


In [53]:
# 
total_totals = total_pop[total_pop['Label (Grouping)'] == 'Total population'].T.reset_index().iloc[:,:]

In [54]:
total_totals

,index,1
0,Label (Grouping),Total population
1,ZCTA5 90001!!Estimate,"59,832"
2,ZCTA5 90001!!Margin of Error,"±1,842"
3,ZCTA5 90001!!Percent,"59,832"
4,ZCTA5 90001!!Percent Margin of Error,(X)
...,...,...
1204,ZCTA5 93563!!Percent Margin of Error,(X)
1205,ZCTA5 93591!!Estimate,"6,744"
1206,ZCTA5 93591!!Margin of Error,±779
1207,ZCTA5 93591!!Percent,"6,744"


In [55]:
total_totals.columns = ['subgroup_info', 'total_population']

In [56]:
total_totals = total_totals[total_totals['subgroup_info'].str.contains('Estimate')]

In [57]:
total_totals['subgroup_info']

1       ZCTA5 90001!!Estimate
5       ZCTA5 90002!!Estimate
9       ZCTA5 90003!!Estimate
13      ZCTA5 90004!!Estimate
17      ZCTA5 90005!!Estimate
                ...          
1189    ZCTA5 93552!!Estimate
1193    ZCTA5 93553!!Estimate
1197    ZCTA5 93560!!Estimate
1201    ZCTA5 93563!!Estimate
1205    ZCTA5 93591!!Estimate
Name: subgroup_info, Length: 302, dtype: object

In [58]:
# Grabs the zipcodes from each string object in a list
# [(lambda a: re.findall(r'[0-9]{5}', a))(z)[0] for z in zip_codes[1:]]

In [59]:
total_zct = [(lambda a: re.findall(r'[0-9]{5}', a))(z)[0] for z in total_totals['subgroup_info']]

In [60]:
# check length
print(len(total_totals['subgroup_info']),len(total_zct))

302 302


In [61]:
total_totals['zipcode'] = total_zct

C:\Users\Allen\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [62]:
total_totals['zipcode'] = total_totals['zipcode'].astype('float')

C:\Users\Allen\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
total_totals = total_totals.drop('subgroup_info', axis = 1)

In [64]:
total_totals

,total_population,zipcode
1,"59,832",90001.0
5,"53,302",90002.0
9,"73,730",90003.0
13,"60,541",90004.0
17,"39,732",90005.0
...,...,...
1189,"39,905",93552.0
1193,"1,709",93553.0
1197,"21,756",93560.0
1201,357.0,93563.0


In [65]:
total_totals['zipcode'].astype('float')

1       90001.0
5       90002.0
9       90003.0
13      90004.0
17      90005.0
         ...   
1189    93552.0
1193    93553.0
1197    93560.0
1201    93563.0
1205    93591.0
Name: zipcode, Length: 302, dtype: float64

In [66]:
final = data_grouped_confidential.reset_index()

NameError: name 'data_grouped_confidential' is not defined

In [ ]:
# merge population data into one data frame
final = final.merge(total_totals, how = 'left', left_on = 'Zip', right_on= 'zipcode').drop('zipcode', axis =1)
final = final.merge(black_totals, how = 'left', left_on = 'Zip', right_on= 'zipcode').drop('zipcode', axis =1)
final = final.merge(hispanic_totals, how = 'left', left_on = 'Zip', right_on= 'zipcode').drop('zipcode', axis =1)

In [ ]:
final['black_population']

In [ ]:
# handle commas in population data so they can be turned into numeric values
final['total_population'] = final['total_population'].astype(str)
final['black_population'] = final['black_population'].astype(str)
final['hispanic_population'] = final['hispanic_population'].astype(str)

final['total_population'] = final['total_population'].str.replace(',', '').astype(float)
final['black_population'] = final['black_population'].str.replace(',', '').astype(float)
final['hispanic_population'] = final['hispanic_population'].str.replace(',', '').astype(float)

In [ ]:
# create percentage values for every zipcode
final['black_percent'] = final['black_population'] / final['total_population']
final['hispanic_percent'] = final['hispanic_population'] / final['total_population']

In [ ]:
final.drop('level_0', axis = 1, inplace = True)

In [ ]:
final

## Data export

In [ ]:
# final.to_csv('PACE_data.csv')

In [ ]:
final.describe()

# Color filter tableau
I wanted to make it so that tableau would adjust the color filter so I'm creating a new dataset that will work for said purpose. Essentially by creating a single percent variable and then stacking two duplicate datasets: one with black data and one with hispanic data. Tableau can color by percent and filter based off the category thus allowing for one to see the differences in real time by changing the filter.

In [ ]:
black_cat = final.drop(['hispanic_percent', 'hispanic_population'],axis = 1).rename(columns = {'black_population' : 'population', 'black_percent' : 'percent'})
black_cat['Category'] = 'Black'

hispanic_cat = final.drop(['black_percent', 'black_population'],axis = 1).rename(columns = {'hispanic_population' : 'population', 'hispanic_percent' : 'percent'})
hispanic_cat['Category'] = 'Hispanic'

none_cat = final.drop(['hispanic_percent', 'hispanic_population', 'black_percent', 'black_population'], axis = 1)
none_cat['Category'] = 'None'
none_cat['population'] = float('NaN')
none_cat['percent'] = 0

PACE_tableau = pd.concat([black_cat, hispanic_cat, none_cat], axis = 0)

In [ ]:
hispanic_cat

In [ ]:
black_cat

## Tableau adjusted data export

In [ ]:
# PACE_tableau.to_csv('PACE_tableau.csv')

# Zip Code Map Layer Tableau
### Created a dataframe with only each zipcode and the population data for all demographics for the purpose of creating a map layer

In [67]:
total_totals

,total_population,zipcode
1,"59,832",90001.0
5,"53,302",90002.0
9,"73,730",90003.0
13,"60,541",90004.0
17,"39,732",90005.0
...,...,...
1189,"39,905",93552.0
1193,"1,709",93553.0
1197,"21,756",93560.0
1201,357.0,93563.0


In [95]:
Zip = pd.DataFrame(final['Zip'].unique())

In [96]:
Zip.columns = ['Zip']

In [97]:
Zip = Zip.merge(total_totals, how = 'left', left_on = 'Zip', right_on = 'zipcode').drop('zipcode', axis = 1)
Zip = Zip.merge(black_totals, how = 'left', left_on = 'Zip', right_on = 'zipcode').drop('zipcode', axis = 1)
Zip = Zip.merge(hispanic_totals, how = 'left', left_on = 'Zip', right_on = 'zipcode').drop('zipcode', axis = 1)
Zip = Zip.dropna(axis = 0)

In [99]:
Zip['total_population'] = Zip['total_population'].astype(str)
Zip['black_population'] = Zip['black_population'].astype(str)
Zip['hispanic_population'] = Zip['hispanic_population'].astype(str)

Zip['total_population'] = Zip['total_population'].str.replace(',', '').astype(float)
Zip['black_population'] = Zip['black_population'].str.replace(',', '').astype(float)
Zip['hispanic_population'] = Zip['hispanic_population'].str.replace(',', '').astype(float)

In [101]:
Zip['black_percent'] = Zip['black_population'] / Zip['total_population']
Zip['hispanic_percent'] = Zip['hispanic_population'] / Zip['total_population']

In [100]:
Zip

,Zip,total_population,black_population,hispanic_population
0,91304.0,54361.0,2709.0,22303.0
1,91311.0,40839.0,2159.0,10550.0
3,91307.0,26341.0,995.0,3810.0
4,91303.0,31586.0,967.0,20279.0
5,91367.0,45970.0,1932.0,5911.0
...,...,...,...,...
260,91731.0,30222.0,118.0,19391.0
261,91702.0,62066.0,2014.0,41612.0
262,91741.0,26794.0,228.0,7202.0
263,91740.0,25877.0,769.0,10895.0


In [102]:
Zip

,Zip,total_population,black_population,hispanic_population,black_percent,hispanic_percent
0,91304.0,54361.0,2709.0,22303.0,0.049834,0.410276
1,91311.0,40839.0,2159.0,10550.0,0.052866,0.258331
3,91307.0,26341.0,995.0,3810.0,0.037774,0.144641
4,91303.0,31586.0,967.0,20279.0,0.030615,0.642025
5,91367.0,45970.0,1932.0,5911.0,0.042027,0.128584
...,...,...,...,...,...,...
260,91731.0,30222.0,118.0,19391.0,0.003904,0.641619
261,91702.0,62066.0,2014.0,41612.0,0.032449,0.670448
262,91741.0,26794.0,228.0,7202.0,0.008509,0.268792
263,91740.0,25877.0,769.0,10895.0,0.029718,0.421030


## Zipcode data export

In [103]:
# Zip.to_csv('Zip_demo_data.csv')